# Solving Frozen Lake Problem Using Value Iteration

## Goal:

Imagine, there is a frozen lake from your home to office, you should walk on the frozen lake
to reach your office. But oops! there will be a hole in the frozen lake in between, so you have
to be careful while walking in the frozen lake to avoid getting trapped at holes.
Look at the below figure where, 

1. S is the starting position (Home)
2. F is the Frozen lake where you can walk
3. H is the Hole which you have to be so careful about
4. G is the Goal (office)

![title](images/B09792_03_50.png)

 Okay, now let us use our agent instead of you to find the correct way to reach the office.
The agent goal is to find the optimal path to reach from S to G without getting trapped at H.
How an agent can achieve this? We give +1 point as a reward to the agent if it correctly
walks on the frozen lake and 0 points if it falls into the hole. So that agent could determine
which is the right action. An agent will now try to find the optimal policy. Optimal policy
implies taking the correct path which maximizes the agent reward. If the agent is
maximizing the reward, apparently agent is learning to skip the hole and reach the
destination.

First, we import necessary libraries

In [1]:
import gymnasium as gym
print(f'Gymnasium v{gym.__version__}')

import numpy as np
print(f'NumPy v{np.__version__}')

Gymnasium v0.26.2
NumPy v1.22.3


Initialize our gym environment

In [2]:
env = gym.make('FrozenLake-v1', render_mode="ansi", is_slippery=False)

 Let us see how the environment looks like

In [3]:
env.reset()
print(env.render())


SFFF
FHFH
FFFH
HFFG



`env.P` below can be used to see the relevant states and rewards for each action taken in that particular state.

See: https://github.com/Farama-Foundation/Gymnasium/blob/1fb28ebe7fdafdd9fdf6998536516ef5171109f6/gymnasium/envs/toy_text/frozen_lake.py#L239C68-L239C68

Each tuple is a form of (transition probability, new state, reward, terminated).

Letting this as `(p, s, r, t)`, return value of `step()` is `(int(s), r, t, False, {"prob":p})`.

In [4]:
env.P[15]

{0: [(1.0, 15, 0, True)],
 1: [(1.0, 15, 0, True)],
 2: [(1.0, 15, 0, True)],
 3: [(1.0, 15, 0, True)]}

In [5]:
num_state = env.observation_space.n
num_action = env.action_space.n

## Value Iteration

Now we define the function called `value_iteration` which performs value iteraion i.e it returns the optimal value
function (value table).

In [6]:
def value_iteration(env, gamma = 1.0):
    
    # initialize value table with zeros
    value_table = np.zeros(num_state)
    
    # set number of iterations and threshold
    no_of_iterations = 100000
    threshold = 1e-20
    
    for i in range(no_of_iterations):
        
        # On each iteration, copy the value table to the updated_value_table
        updated_value_table = np.copy(value_table)
        
        # Now we calculate Q Value for each actions in the state 
        # and update the value of a state with maximum Q value
        
        for state in range(num_state):  # Calculate V(s) for each s
            Q_value = np.zeros(num_action)

            for action in range(num_action):    # Calculate Q(s,a) for each a | given s.
                next_states_rewards = 0
                for next_sr in env.P[state][action]:    # Calculate P(R+rV(s')) for each of n possible next states s' | given s & a. (In this environment, n=3 for S or F, n=1 for G or H.)
                    trans_prob, next_state, reward, _ = next_sr
                    next_states_rewards += trans_prob * (reward + gamma * updated_value_table[next_state])
                
                Q_value[action] = next_states_rewards   # Q(s,a)|s,a = sum{P(R+rV(s'))}
                
            value_table[state] = max(Q_value)   # V(s)|s = max_a{Q(s,a)}
            
        # we will check whether we have reached the convergence i.e whether the difference 
        # between our value table and updated value table is very small. But how do we know it is very
        # small? We set some threshold and then we will see if the difference is less
        # than our threshold, if it is less, we break the loop and return the value function as optimal
        # value function
        
        # if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
        #     print ('Value-iteration converged at iteration #%d.' %(i+1))
        #     break
    
    return value_table

 Now, we define a function called `extract_policy` for extracting optimal policy from the optimal value function. 
i.e We calculate Q value using our optimal value function and pick up
the actions which has the highest Q value for each state as the optimal policy.

In [7]:
def extract_policy(value_table, gamma = 1.0):
 
    # initialize the policy with zeros
    policy = np.zeros(num_state) 
    
    for state in range(num_state):
        
        # initialize the Q table for a state
        Q_table = np.zeros(num_action)
        
        # compute Q value for all ations in the state
        for action in range(num_action):
            for next_sr in env.P[state][action]: 
                trans_prob, next_state, reward, _ = next_sr 
                Q_table[action] += (trans_prob * (reward + gamma * value_table[next_state]))
        
        # select the action which has maximum Q value as an optimal action of the state
        policy[state] = np.argmax(Q_table)
    
    return policy

First, We compute the optimal value function

In [8]:
optimal_value_function = value_iteration(env=env,gamma=0.9)

print()
with np.printoptions(precision=4, suppress=True):
    for i in range(4):
        print(optimal_value_function[4*i : 4*(i+1)])


[0.5905 0.6561 0.729  0.6561]
[0.6561 0.     0.81   0.    ]
[0.729 0.81  0.9   0.   ]
[0.  0.9 1.  0. ]


and we derive the optimal policy from the optimal value function

In [9]:
optimal_policy = extract_policy(optimal_value_function, gamma=1.0)

action = {0:'<<', 1:'vv', 2:'>>', 3:'^^'}
arrows = ''
for i in range(num_state):
    arrows += action[optimal_policy[i]]+', '
print(arrows[:-2])

vv, >>, vv, <<, vv, <<, vv, <<, >>, vv, vv, <<, <<, >>, vv, <<


- Note 1. Value iteration for Frozen Lake doesn't seem good. The value of the goal state remains 0 while the values of the other states keep increasing, which makes the last step for success will never be made.
- Note 2. If you set `is_slippery=True` for the environment or set `gamma=1.0` at `value_iteration()`, the results will be more bad.